In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

In [2]:
# This is needed to display the images.
%matplotlib inline

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

In [4]:
MODEL_NAME = 'inference_graph'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = 'training/label_map.pbtxt'

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [6]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


In [7]:
# PATH_TO_LABELS = 'training/label_map.pbtxt'
# category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [8]:
def run_inference_for_single_image(image, graph):
    if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

    # Run inference
    output_dict = sess.run(tensor_dict,
                            feed_dict={image_tensor: np.expand_dims(image, 0)})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
        'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [9]:
def show_inference(detection_graph, image_path):
    try:
        with detection_graph.as_default():
            with tf.Session() as sess:
                    # Get handles to input and output tensors
                    ops = tf.get_default_graph().get_operations()
                    all_tensor_names = {output.name for op in ops for output in op.outputs}
                    tensor_dict = {}
                    for key in [
                      'num_detections', 'detection_boxes', 'detection_scores',
                      'detection_classes', 'detection_masks'
                    ]:
                        tensor_name = key + ':0'
                        if tensor_name in all_tensor_names:
                            tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                          tensor_name)
                    
                        
                        # the array based representation of the image will be used later in order to prepare the
                        # result image with boxes and labels on it.
                        image_np = np.array(Image.open(image_path))
                        # Actual detection.
                        output_dict = run_inference_for_single_image(detection_graph, image_np)
                        # Visualization of the results of a detection.
                        vis_util.visualize_boxes_and_labels_on_image_array(
                          image_np,
                          output_dict['detection_boxes'],
                          output_dict['detection_classes'],
                          output_dict['detection_scores'],
                          category_index,
                          instance_masks=output_dict.get('detection_masks_reframed', None),
                          use_normalized_coordinates=True,
                          line_thickness=8)

                        display(Image.fromarray(image_np))
                          
    except Exception as e:
        print(e)

In [10]:
show_inference(detection_graph,'test_images/goose.jpg')

name 'tensor_dict' is not defined


In [11]:
# img = cv2.imread('/test_images/goose.jpg')

In [12]:
img = np.array(Image.open('test_images/goose.jpg'))

In [13]:
import glob, os

In [14]:
img_print = Image.fromarray(img, 'RGB')
img_print.show()

In [15]:
test_images = glob.glob("test_images/*.jpg")

In [16]:
print(test_images)

['test_images\\goose.jpg', 'test_images\\image1.jpg', 'test_images\\image2.jpg']


In [17]:
show_inference()

TypeError: show_inference() missing 2 required positional arguments: 'detection_graph' and 'image_path'

In [ ]:
# def run_inference_for_single_image(model, image):
#     image = np.asarray(image)
#     # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
#     input_tensor = tf.convert_to_tensor(image)
#     # The model expects a batch of images, so add an axis with `tf.newaxis`.
#     input_tensor = input_tensor[tf.newaxis,...]

#     # Run inference
#     output_dict = model(input_tensor)

#     # All outputs are batches tensors.
#     # Convert to numpy arrays, and take index [0] to remove the batch dimension.
#     # We're only interested in the first num_detections.
#     num_detections = int(output_dict.pop('num_detections'))
#     output_dict = {key:value[0, :num_detections].numpy() 
#                  for key,value in output_dict.items()}
#     output_dict['num_detections'] = num_detections

#     # detection_classes should be ints.
#     output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

#     # Handle models with masks:
#     if 'detection_masks' in output_dict:
#     # Reframe the the bbox mask to the image size.
#     detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
#               output_dict['detection_masks'], output_dict['detection_boxes'],
#                image.shape[0], image.shape[1])      
#     detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
#                                        tf.uint8)
#     output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
#     return output_dict

In [ ]:
# def image_detect(image):
#     try:
#     with detection_graph.as_default():
#         with tf.Session() as sess:
#                 # Get handles to input and output tensors
#                 ops = tf.get_default_graph().get_operations()
#                 all_tensor_names = {output.name for op in ops for output in op.outputs}
#                 tensor_dict = {}
#                 for key in [
#                   'num_detections', 'detection_boxes', 'detection_scores',
#                   'detection_classes', 'detection_masks'
#                 ]:
#                     tensor_name = key + ':0'
#                     if tensor_name in all_tensor_names:
#                         tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
#                       tensor_name)
                        
#     run_inference_for_single_image(img, detection_graph)

In [ ]:
break

In [ ]:
import cv2
cap = cv2.VideoCapture(0)
try:
    with detection_graph.as_default():
        with tf.Session() as sess:
                # Get handles to input and output tensors
                ops = tf.get_default_graph().get_operations()
                all_tensor_names = {output.name for op in ops for output in op.outputs}
                tensor_dict = {}
                for key in [
                  'num_detections', 'detection_boxes', 'detection_scores',
                  'detection_classes', 'detection_masks'
                ]:
                    tensor_name = key + ':0'
                    if tensor_name in all_tensor_names:
                        tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                      tensor_name)

                while True:
                    ret, image_np = cap.read()
                    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                    image_np_expanded = np.expand_dims(image_np, axis=0)
                    # Actual detection.
                    output_dict = run_inference_for_single_image(image_np, detection_graph)
                    # Visualization of the results of a detection.
                    vis_util.visualize_boxes_and_labels_on_image_array(
                        image_np,
                        output_dict['detection_boxes'],
                        output_dict['detection_classes'],
                        output_dict['detection_scores'],
                        category_index,
                        instance_masks=output_dict.get('detection_masks'),
                        use_normalized_coordinates=True,
                        line_thickness=8)
                    cv2.imshow('object_detection', cv2.resize(image_np, (800, 600)))
                    if cv2.waitKey(25) & 0xFF == ord('q'):
                        cap.release()
                        cv2.destroyAllWindows()
                        break
except Exception as e:
    print(e)
    cap.release()

In [ ]:
import cv2
cap = cv2.imread('test_images/goose.jpg')
try:
    with detection_graph.as_default():
        with tf.Session() as sess:
                # Get handles to input and output tensors
                ops = tf.get_default_graph().get_operations()
                all_tensor_names = {output.name for op in ops for output in op.outputs}
                tensor_dict = {}
                for key in [
                  'num_detections', 'detection_boxes', 'detection_scores',
                  'detection_classes', 'detection_masks'
                ]:
                    tensor_name = key + ':0'
                    if tensor_name in all_tensor_names:
                        tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                      tensor_name)

#                 while True:
                image_np = cap
                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image_np_expanded = np.expand_dims(image_np, axis=0)
                # Actual detection.
                output_dict = run_inference_for_single_image(image_np, detection_graph)
                # Visualization of the results of a detection.
                vis_util.visualize_boxes_and_labels_on_image_array(
                    image_np,
                    output_dict['detection_boxes'],
                    output_dict['detection_classes'],
                    output_dict['detection_scores'],
                    category_index,
                    instance_masks=output_dict.get('detection_masks'),
                    use_normalized_coordinates=True,
                    line_thickness=8)
                cv2.imshow('object_detection', cv2.resize(image_np, (800, 600)))
                cv2.waitKey(0)
#                 if cv2.waitKey(25) & 0xFF == ord('q'):
# #                         cap.release()
#                     cv2.destroyAllWindows()
# #                     break
except Exception as e:
    print(e)
#     cap.release()